In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from celluloid import Camera
from IPython.display import HTML

from typing import Literal

from turing_recursion.lib.turing import TuringMachine

In [ ]:
"""
Implementation of the same game of life as in `turing_periodic` but with a clockwise turing machine, i.e. we assume that the tape is circular,
which just means, that if the head moves beyond the edge of the tape, it just ends up on the other side.

While we're at it -> we are checking to see if it's possible to make the machine a little more efficient
the previous implementation takes around 70k steps for a 10x10 board.
"""

In [3]:
# Turing machine code-generator functions

symbols = [
    0, 1,
    '00', '01', '10', '11', # intermediate states (current/next)
    # marked cells
    'X', 'XO', 'XX',
    # marked center cells
    'CO', 'CX',
    # tape boundaries
    '^', '$',
    # grid boundaries
    'E', 'R'
]

In [4]:
# Simplified code-gen functions, with less edge-case handling
# aka more manual handling, but less states and complexity
# and they're ready for clock-wise operation

def _tm_jmp(name: str, size: int, end_state: str, expected_symbols: list[int | str] | None = None):
    """
    another relative jump for a turing machine
    but it only does right-jumps and it jumps beyond the tape (only use this for boundary_condition='wrapping')
    """
    assert size > 0, "size must be positive"
    states = {}

    expected_symbols = expected_symbols or symbols

    for i in range(size - 1):
        state_name = f'{name}_{i}' if i > 0 else name
        for s in expected_symbols:
            states[(state_name, s)] = (s, 'R', f'{name}_{i+1}')
    for s in expected_symbols:
        states[(f'{name}_{size - 1}', s)] = (s, 'R', end_state)
    return states

def _tm_jmp_explicit_wrapping(
    name: str,
    size: int,
    end_state: str,
    expected_symbols: list[int | str] | None = None,
    skip: int = 4
):
    """
    yet another relative jump for a turing machine
    but if this encounters the grid-end symbol (E), it adds 4 to the jump-size
    this allows writing code that basically considers only the grid-cells, with wrap-around, 
    without worrying about conditionally adding logic to jump around the tail-state

    note: the tail-symbols must be manually included.
    """
    assert size > 0, "size must be positive"

    expected_symbols = [ s for s in (expected_symbols or symbols) if s != 'E' ]

    states = {}
    for i in range(size - 1 + skip):
        index = i - skip

        state_name = f'{name}_{index}' if abs(index) > 0 else name
        next_state_name = f'{name}_{index + 1}' if index != -1 else name
        next_state_wrap_name = f'{name}_{index - skip}' if index != skip else name

        for s in expected_symbols:
            states[(state_name, s)] = (s, 'R', next_state_name)
        states[(state_name, 'E')] = ('E', 'R', next_state_wrap_name)

    state_name = f'{name}_{size - 1}' if size > 1 else name
    for s in expected_symbols:
        states[(state_name, s)] = (s, 'R', end_state)
    states[(state_name, 'E')] = ('E', 'R', f'{name}_{size - 1 - skip}' if size != skip else name)
    return states

def _tm_symjmp(
    name: str,
    symbol: int | str | list[int | str],
    end_state: str,
    expected_symbols: list[int | str] | None = None
):
    """
    jumps forward until it hits a certain symbol, stops at one after the symbol
    """
    expected_symbols = expected_symbols or symbols

    stop_symbols = symbol if isinstance(symbol, list) else [symbol]
    skip_symbols = [s for s in symbols if s not in stop_symbols] 

    states = {
        **{(f'{name}', s): (s, 'R', f'{name}') for s in skip_symbols},
        **{(f'{name}', s): (s, 'R', end_state) for s in stop_symbols},
    }
    return states

def _tm_increment_reverse(name: str, bit_size: int, end_state: str):
    """
    increments a binary number with bit_size bits
    starts at the number's least significant bit, and assumes LSB first storage (LSB is the leftmost bit)
    does not handle overflow - i.e. the number rolls over to 0
    starting-state:
        {name}_0
    ending-state:
        {name}_end, one position after the end of the number (to the right)
    """
    states = {
        **_tm_jmp(f'{name}_ret', bit_size - 1, end_state, expected_symbols=[0, 1]),
    }

    for i in range(bit_size):
        state_name = f"{name}_{i}" if i > 0 else name
        return_state_name = f"{name}_ret_{i}"
        if i == 0:
            return_state_name = f"{name}_ret"
        elif i == bit_size - 1:
            return_state_name = end_state
        
        # if the current bit is 0, set it to 1
        # and just jump to the end
        states[(state_name, 0)] = (1, 'R', return_state_name)

        # if the current bit is 1, set it to 0 and move to the next bit
        # unless it's the last bit, in which case we're done
        if i == bit_size - 1:
            states[(state_name, 1)] = (0, 'R', return_state_name)
        else:
            states[(state_name, 1)] = (0, 'R', f"{name}_{i + 1}")
    return states

def _tm_any_of_transition(from_state: str, to_state: str, allowed_symbols: list[int | str]):
    """
    Like tm_any_transition, but with defined states only
    and clockwise only
    """
    states = {
        (from_state, s): (s, 'R', to_state) for s in allowed_symbols
    }
    return states

def _tm_mark(state: str, next_state: str):
    states = {
        # not doing the marking for unset cells
        (state, 0): (0, 'R', next_state),
        (state, '00'): ('00', 'R', next_state),
        (state, '01'): ('01', 'R', next_state),
        # but marking the set cells
        (state, 1): ('X', 'R', next_state),
        (state, '10'): ('XO', 'R', next_state),
        (state, '11'): ('XX', 'R', next_state),
    }
    return states

In [ ]:
# Tests for the tm-codegen functions

# Implicitly tests the relative jump function as well
def test_tm_increment_reverse(size: int):
    for initial in tqdm(range(2 ** size), desc="Testing increment"):
        name = 'inc'
        initial_bitstring = bin(initial)[2:].zfill(size)[::-1]
        # print(f"Initial: {initial_bitstring}")

        tape = [int(e) for e in initial_bitstring] + ['$']
        blank_symbol = ' '
        initial_state = name
        end_state = f'{name}_end'
        final_states = {f'{name}_end'}
        
        transition_function = {
            **_tm_increment_reverse(name, size, end_state),
        }
        # for state, symbol in transition_function:
        #     print(f"{state}: {symbol} -> {transition_function[(state, symbol)]}")


        tm = TuringMachine(tape, blank_symbol, initial_state, final_states, transition_function)
        required_steps = tm.run(debug=False)
        # print(f"Initial: {initial}, Steps: {required_steps}")

        result = int(''.join(map(str, tm.tape[:-1][::-1])), 2)
        # print(f"Initial: {initial}, Result: {result}")
        expected = initial + 1 if initial < 2 ** size - 1 else 0
        assert result == expected, f"Given: {initial}, Expected: {expected}, Got: {result}"
        assert tm.current_state == end_state, f"Expected final state to be '{name}_end', got {tm.current_state}"
        assert tm.head_position == len(tape) - 1, f"Expected head position to be at the end of the tape, got {tm.head_position}"
    
    print(f"Turing machine increment test passed, for size {size}")

test_tm_increment_reverse(4)

In [6]:
def create_tm_gol_tape(size: int, grid: list[int]) -> list[str | int]:
    tape = ['^']
    for i in range(size):
        row = grid[i * size:(i + 1) * size]
        tape += row
        if i < size - 1:
            tape.append('R')
    tape += ['E', 0, 0, 0, '$']
    return tape

def visualize_tm_gol_state(tape: list[str | int], size: int, show: bool = False) -> list[int]:
    grid = []
    for i in range(size):
        row = tape[1 + i * (size + 1):1 + (i + 1) * (size + 1)][:-1]
        grid += row

    grid = np.array(grid).reshape(size, size)
    plt.imshow(1.0 - grid, cmap='gray')
    for i in range(size + 1):
        plt.axhline(i - 0.5, color='black', lw=0.5)
        plt.axvline(i - 0.5, color='black', lw=0.5)
    if show:
        plt.show()

def visualize_tm_gol_state_intermediate(tape: list[str | int], size: int, show: bool = False) -> list[int]:
    grid = []
    for i in range(size):
        row = tape[1 + i * (size + 1):1 + (i + 1) * (size + 1)][:-1]
        # replace stuff in the row
        row = [[1.0, 1.0, 1.0] if cell in [0, '00', '01'] else cell for cell in row] # replace off states with white
        row = [[0.0, 0.0, 0.0] if cell in [1, '10', '11'] else cell for cell in row] # replace on states with black
        row = [[0.0, 0.0, 1.0] if cell in ['CO', 'CX'] else cell for cell in row] # replace center cells with blue
        row = [[1.0, 0.0, 0.0] if cell in ['X', 'XX', 'XO'] else cell for cell in row] # replace marked, and on cells with red
        row = [[0.0, 1.0, 0.0] if cell in ['O', 'OO', 'OX'] else cell for cell in row] # replace marked, and off cells with green
        grid += row
    grid = np.array(grid).reshape(size, size, 3)
    plt.imshow(grid)
    for i in range(size + 1):
        plt.axhline(i - 0.5, color='black', lw=0.5)
        plt.axvline(i - 0.5, color='black', lw=0.5)
    if show:
        plt.show()

In [7]:
# X replaces a 1, O replaces a 0, CX, CO replace the center cell

# cell neighborhood:
# 0 1 2 n
# 3 X 4 n
# 5 6 7 n

# we need better edge-detection
# if the symbol before C is ^ or R, we're at the left edge of the grid
# if the symbol after C is E or R, we're at the right edge of the grid

# the other two cases (top and bottom) are a bit more tricky
# basically, there's no good way, except a fwd and bwd-jump to one the start/end of the row

# baseline version requires ~70k steps for ea. update on a 10x10 grid
# we're already down to ~53k steps with the new version
# and better unmark code brought us down to ~50k
# purely clockwise motion brought it back up to 61754 cycles
# removing marking for 0-neighbors brought it down to 38754 cycles


def _make_tm_gol(size: int):
    states = {
        # main entry-point
            # call at the beginning of the tape (at the ^ symbol)
            ('__entry_point', '^'): ('^', 'R', 'nb_mark_start'),

        # fn nb_mark_start
            # if we want to get rid of the left-moves, we need to use a different order:
            # x 4, 567, 012, 3

            # skip over the row-end marker
            ('nb_mark_start', 'R'): ('R', 'R', 'nb_mark_start'),
            ('nb_mark_start', 'E'): ('E', 'R', '__call_cell_collapse'),

            # mark the center cell
            ('nb_mark_start', 0): ('CO', 'R', 'nb_mark_4'),
            ('nb_mark_start', 1): ('CX', 'R', 'nb_mark_4'),

            # cell 4
            **_tm_mark('nb_mark_4', 'nb_mark_5_jmp'),

            # if cell 4 is out of bounds
                ('nb_mark_4', 'R'): ('R', 'R', 'nb_mark_4_wrap_7'),
                ('nb_mark_4', '^'): ('^', 'R', 'nb_mark_4_wrap_7'),

                # cell 7
                **_tm_mark('nb_mark_4_wrap_7', 'nb_mark_4_wrap_5_jmp'),

                # jump to cell 5
                **_tm_jmp('nb_mark_4_wrap_5_jmp', (size - 1) - 2, 'nb_mark_4_wrap_5'),

                # cell 5
                **_tm_mark('nb_mark_4_wrap_5', 'nb_mark_4_wrap_6'),

                # cell 6
                **_tm_mark('nb_mark_4_wrap_6', 'nb_mark_4_wrap_0_jmp'),

                # jump to cell 0
                **_tm_jmp_explicit_wrapping('nb_mark_4_wrap_0_jmp', ((size - 3) * (size + 1)) + 1, 'nb_mark_4_wrap_0'),

                # cell 0
                **_tm_mark('nb_mark_4_wrap_0', 'nb_mark_4_wrap_1_jmp'),

                # jump to cell 1
                **_tm_jmp('nb_mark_4_wrap_1_jmp', (size - 1) - 2, 'nb_mark_4_wrap_1'),

                # cell 1
                **_tm_mark('nb_mark_4_wrap_1', 'nb_mark_4_wrap_2'),

                # cell 2
                **_tm_mark('nb_mark_4_wrap_2', 'nb_mark_4_wrap_4_jmp'),

                # jump back to the beginning and mark 4, then 3
                **_tm_jmp_explicit_wrapping('nb_mark_4_wrap_4_jmp', 1, 'nb_mark_4_wrap_4'),

                # cell 4
                **_tm_mark('nb_mark_4_wrap_4', 'nb_mark_4_wrap_3_jmp'),

                # jump to cell 3
                **_tm_jmp('nb_mark_4_wrap_3_jmp', (size - 1) - 2, 'nb_mark_3'),

            # cell 4 is oob, but on the rb edge
            ('nb_mark_4', 'E'): ('E', 'R', 'nb_mark_4_jmp_a'),
            **_tm_jmp_explicit_wrapping('nb_mark_4_jmp_a', 4, 'nb_mark_4'),


            # jump to cell 5
            **_tm_jmp_explicit_wrapping('nb_mark_5_jmp', (size - 1) - 1, 'nb_mark_5'), # jump by one row and back by one
            **_tm_mark('nb_mark_5', 'nb_mark_6'),

            # if cell 5 is out of bounds
                ('nb_mark_5', 'R'): ('R', 'R', 'nb_mark_6_wrap_5'),
                ('nb_mark_5', '^'): ('^', 'R', 'nb_mark_6_wrap_5'),

                # cell 6
                **_tm_mark('nb_mark_6_wrap_5', 'nb_mark_7_wrap_5'),

                # cell 7
                **_tm_mark('nb_mark_7_wrap_5', 'nb_mark_5_wrap_5_jmp'),

                # jump to end of the row
                **_tm_jmp('nb_mark_5_wrap_5_jmp', (size - 1) - 2, 'nb_mark_5_wrap_5'), # jump by one row and back by 2

                # cell 5
                **_tm_mark('nb_mark_5_wrap_5', 'nb_mark_5_wrap_0_jmp'),

                # jump to cell 0
                **_tm_jmp_explicit_wrapping('nb_mark_5_wrap_0_jmp', (size * (size - 1) - 1) - ((size - 1) - 2) - 5, 'nb_mark_0'),

            # cell 5 is oob, but on the rb edge
            ('nb_mark_5', 'E'): ('E', 'R', 'nb_mark_5_jmp_a'),
            **_tm_jmp_explicit_wrapping('nb_mark_5_jmp_a', 4, 'nb_mark_5'),

            # cell 6
            **_tm_mark('nb_mark_6', 'nb_mark_7'),
            
            # cell 7
            **_tm_mark('nb_mark_7', 'nb_mark_0_jmp'),
            

            # jump to cell 0
            **_tm_jmp_explicit_wrapping('nb_mark_0_jmp', ((size - 2) * (size + 1)) - 3, 'nb_mark_0', skip=4),

            # cell 0
            **_tm_mark('nb_mark_0', 'nb_mark_1'),

            # ensure that we treat the E symbol the same as if it were to wrap around to the ^ symbol
            ('nb_mark_0', 'E'): ('E', 'R', 'nb_mark_0_jmp_a'),
            **_tm_jmp_explicit_wrapping('nb_mark_0_jmp_a', 4, 'nb_mark_0'),


            # if cell 0 is out of bounds
                ('nb_mark_0', 'R'): ('R', 'R', 'nb_mark_1_wrap_0'),
                ('nb_mark_0', '^'): ('^', 'R', 'nb_mark_1_wrap_0'),

                # cell 1
                **_tm_mark('nb_mark_1_wrap_0', 'nb_mark_2_wrap_0'),

                # cell 2
                **_tm_mark('nb_mark_2_wrap_0', 'nb_mark_0_wrap_0_jmp'),

                # jump to end of the row
                **_tm_jmp('nb_mark_0_wrap_0_jmp', (size - 1) - 2, 'nb_mark_0_wrap_0'), # jump by one row and back by 2

                # cell 0
                **_tm_mark('nb_mark_0_wrap_0', 'nb_mark_3'),

                # jump to cell 3
                ('nb_mark_0_wrap_3_0', 'R'): ('R', 'R', 'nb_mark_3'),
                ('nb_mark_0_wrap_3_0', 'E'): ('E', 'R', 'nb_mark_0_wrap_3_jmp'),
                **_tm_jmp('nb_mark_0_wrap_3_jmp', 4, 'nb_mark_3'),
            # cell 1
            **_tm_mark('nb_mark_1', 'nb_mark_2'),

            # cell 2
            **_tm_mark('nb_mark_2', 'nb_mark_3_jmp'),

            # jump to cell 3
            **_tm_jmp_explicit_wrapping('nb_mark_3_jmp', size - 2, 'nb_mark_3'),

            # cell 3
            # if it was set, we can directly start counting without marking it
            # otherwise we go on to the counting-fn
            **_tm_any_of_transition('nb_mark_3', 'neighbor_count_start', [0, '01', '00']),
            **_tm_any_of_transition('nb_mark_3', 'inc_jmp', [1, '10', '11']),
            # if cell 3 is out of bounds
                **_tm_any_of_transition('nb_mark_3', 'nb_mark_3_wrap_3_jmp', [ '^', 'R' ]),
                **_tm_jmp('nb_mark_3_wrap_3_jmp', size - 1, 'nb_mark_3'), # jump by one row and back by 1

            # if cell 3 is out of bounds, but on the other side, jump to the beginning
            ('nb_mark_3', 'E'): ('E', 'R', 'nb_mark_3_jmp_beginning'),
            **_tm_jmp('nb_mark_3_jmp_beginning', 4, 'nb_mark_3'),


        # fn neighbor_count
            # we need to get to the end *once* because there will not be any set neighbors (or very unlikely at least) in the first fwd-pass
            # this is because we usually start numbering at the lower right corner - which means there are mostly no set neighbors
            # unless there's some wrapping going on
            # once setup, we can just go wrap around the beginning and then start counting from there
            # with the new counting strategy, we don't - i.e. we can start counting from the beginning
            **_tm_any_of_transition('neighbor_count_start', 'neighbor_count_start_jmp', [0, 1, 'CO', 'CX', 'R', 'E']),
            **_tm_symjmp('neighbor_count_start_jmp', ['^'], 'neighbor_count_next'),
            # ('neighbor_count_start', 'X'): (1, 'R', 'inc_jmp'),
            # ('neighbor_count_start', 'XX'): (1, 'R', 'inc_jmp'),
            # ('neighbor_count_start', 'XO'): (1, 'R', 'inc_jmp'),
            # E can happen if the center is at the bottom-left corner
            **_tm_any_of_transition('neighbor_count_next', 'neighbor_count_next', ['$', '^', 'R', 0, 1, '00', '01', '10', '11', 'CO', 'CX']),
            ('neighbor_count_next', 'X'): (1, 'R', 'inc_jmp'),
            ('neighbor_count_next', 'XX'): ('11', 'R', 'inc_jmp'),
            ('neighbor_count_next', 'XO'): ('10', 'R', 'inc_jmp'),
            ('neighbor_count_next', 'E'): ('E', 'R', 'cell_update'), # terminates at E + 1


        # fn inc_jmp
            # function to increment the counter
            # makes use of the fact, that we can store the number in reverse order, which makes traversal much more efficient
            # since we know that this will *always* from any position < E, we can instead jump to E, then forward by 2 (since symjmp end up at E + 1)
            **_tm_symjmp('inc_jmp', ['E'], '__inc_inner_call'),
            **_tm_increment_reverse('__inc_inner_call', 3, 'neighbor_count_next'), # this stops at the $ symbol

        
        # fn cell_update
            # Function to update the cell state
            # must be called at E + 1
            # Check LSB
            ('cell_update', 1): (0, 'R', 'cell_update_count_1_3_5_7'),  # LSB is 1
            ('cell_update', 0): (0, 'R', 'cell_update_count_0_2_4_6'),  # LSB is 0

            # Check LSB + 1
            ('cell_update_count_1_3_5_7', 1): (0, 'R', 'cell_update_count_3_7'),    # LSB + 1 is 1
            ('cell_update_count_1_3_5_7', 0): (0, 'R', 'cell_update_count_1_5'),    # LSB + 1 is 0
            ('cell_update_count_0_2_4_6', 1): (0, 'R', 'cell_update_count_2_6'),    # LSB + 1 is 1
            ('cell_update_count_0_2_4_6', 0): (0, 'R', 'cell_update_count_0_4'),    # LSB + 1 is 0

            # Check LSB + 2
            ('cell_update_count_0_4', 0): (0, 'R', 'cell_update_count_other_jmp'),  # counter = 0, unset
            ('cell_update_count_1_5', 0): (0, 'R', 'cell_update_count_other_jmp'),  # counter = 1, unset
            ('cell_update_count_2_6', 0): (0, 'R', 'cell_update_count_2_jmp'),      # counter = 2, set
            ('cell_update_count_3_7', 0): (0, 'R', 'cell_update_count_3_jmp'),      # counter = 3, set
            ('cell_update_count_0_4', 1): (0, 'R', 'cell_update_count_other_jmp'),  # counter = 4, unset
            ('cell_update_count_1_5', 1): (0, 'R', 'cell_update_count_other_jmp'),  # counter = 5, unset
            ('cell_update_count_2_6', 1): (0, 'R', 'cell_update_count_other_jmp'),  # counter = 6, unset
            ('cell_update_count_3_7', 1): (0, 'R', 'cell_update_count_other_jmp'),  # counter = 7, unset

            # the LSB check should end at the $ symbol, i.e. the end of the tape, so we can jump forwards to the CX/CO symbols
            # these are like _tm_fwd_symjmp, but with integrated end-state handling
            **_tm_any_of_transition('cell_update_count_other_jmp', 'cell_update_count_other_jmp', ['$', '^', 0, 1, '00', '01', '10', '11', 'R']),
            ('cell_update_count_other_jmp', 'CO'): ('00', 'R', 'nb_mark_start'),
            ('cell_update_count_other_jmp', 'CX'): ('10', 'R', 'nb_mark_start'),

            **_tm_any_of_transition('cell_update_count_2_jmp', 'cell_update_count_2_jmp', ['$', '^', 0, 1, '00', '01', '10', '11', 'R']),
            ('cell_update_count_2_jmp', 'CO'): ('00', 'R', 'nb_mark_start'),
            ('cell_update_count_2_jmp', 'CX'): ('11', 'R', 'nb_mark_start'),

            **_tm_any_of_transition('cell_update_count_3_jmp', 'cell_update_count_3_jmp', ['$', '^', 0, 1, '00', '01', '10', '11', 'R']),
            ('cell_update_count_3_jmp', 'CO'): ('01', 'R', 'nb_mark_start'),
            ('cell_update_count_3_jmp', 'CX'): ('11', 'R', 'nb_mark_start'),
        
        # fn cell_collapse
            # Function to collapse the dual-state representations
            # must be called at E + 1
            # jumps back to the beginning of the tape
            # but now, without any left-move-symbols, i.e. we advance from E + 1 to the first symbol
            # the jumps only need to work for 0, since the counter we're jumping through should already have been reset
            **_tm_jmp('__call_cell_collapse', 5, 'cell_collapse', expected_symbols=[0, '$', '^']), # jump through the counter and end of tape back to the first symbol
            ('cell_collapse', '00'): (0, 'R', 'cell_collapse'),
            ('cell_collapse', '01'): (1, 'R', 'cell_collapse'),
            ('cell_collapse', '10'): (0, 'R', 'cell_collapse'),
            ('cell_collapse', '11'): (1, 'R', 'cell_collapse'),
            ('cell_collapse', 'R'): ('R', 'R', 'cell_collapse'),
            # Once we hit the end of the grid, we're done
            ('cell_collapse', 'E'): ('E', 'R', '__ret_cell_collapse'),
            **_tm_jmp('__ret_cell_collapse', 4, '__entry_point', expected_symbols=[0, '$']), # jump back to the beginning of the tape, to the first symbol
    }

    return states

In [ ]:

visualize_intermediates = False

size = 10
state = [
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
    0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
    0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
    0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
]

tape = create_tm_gol_tape(size, state)

blank_symbol = ' '
initial_state = '__entry_point'
final_states = {'__entry_point', 'done'}
if visualize_intermediates:
    final_states.add('neighbor_count_start')

tm = TuringMachine(tape, blank_symbol, initial_state, final_states, _make_tm_gol(size), boundary_condition='wrapping')

fig = plt.figure()
camera = Camera(fig)

visualize_tm_gol_state(tm.tape, size)
camera.snap()

for i in range(50):
    num_steps = 0
    # tm.current_state = '__entry_point'
    num_steps += tm.run(debug=False)

    if tm.current_state == 'neighbor_count_start':
        visualize_tm_gol_state_intermediate(tm.tape, size)
        camera.snap()

    iter = 0
    while tm.current_state != '__entry_point':
        # print(f'running index {iter}')
        num_steps += tm.run(debug=iter in [99], max_steps=5000)
        # print(tm.current_state)

        if tm.current_state == 'cell_update_start':
            # get the index of the CX or CO symbol
            idx = tm.tape.index('CX') if 'CX' in tm.tape else tm.tape.index('CO') - 1
            x_idx = idx % (size + 1)
            y_idx = idx // (size + 1)
            print(tm.tape[-4:-1], (y_idx, x_idx))

        elif tm.current_state in ['cell_update_set_unset', 'cell_update_set_set', 'cell_update_unset_unset', 'cell_update_unset_set']:
            print(tm.current_state)

        elif tm.current_state == 'neighbor_count_start':
            visualize_tm_gol_state_intermediate(tm.tape, size)
            camera.snap()
            iter += 1
            if iter >= 100:
                break
        else:
            print(tm.current_state)
            break
    visualize_tm_gol_state(tm.tape, size)
    camera.snap()
    print(f'step {i} ran for {num_steps} cycles')

animation = camera.animate()
HTML(animation.to_html5_video())

In [ ]:
# Sanity check:
# are there any states that go *left*?

used_symbols = set()
states = _make_tm_gol(10)
for (state, symbol), (write, direction, next_state) in states.items():
    assert direction != 'L', f"State {state} writes {write} and goes left"
    used_symbols.add(symbol)

print(f'Total states: {len(states)}')
print(f'Used symbols: {used_symbols}')